In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 #imports

In [ ]:
# needed imports
from matplotlib import pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader
from torchvision import models,transforms, utils
from torch.utils.data import random_split
from torch.nn import Conv2d
from torch import flatten
from torch.optim import Adam
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
import torch.nn.functional as F
from torch.nn import Module
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
import argparse
import time
import torch
import cv2
import os


In [ ]:
#  define the configuration 
class Config:
    # specify image dimension
    IMAGE_SIZE = 224

    # specify ImageNet mean and standard deviation
    #MEAN = [0.485, 0.456, 0.406]
    #STD = [0.229, 0.224, 0.225]

    # determine the device we will be using for inference
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # specify path to the ImageNet labels
    IN_LABELS = "ilsvrc2012_wordnet_lemmas.txt"

# initialize a config object
config = Config()

In [ ]:
#  define the functions 
def plt_imshow(title, image):
    # convert the image frame BGR to RGB color space and display it
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #print(image.shape)
    plt.imshow(image)
    plt.title(title)
    plt.grid(False)
    plt.show()
    
# preprocess the images     
def preprocess_image(image):
    # swap the color channels from BGR to RGB, resize it, and scale
    # the pixel values to [0, 1] range
    image = cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (config.IMAGE_SIZE, config.IMAGE_SIZE))
    image = image.astype("float32") / 255.0

    # subtract ImageNet mean, divide by ImageNet standard deviation,
    # set "channels first" ordering, and add a batch dimension
    #image -= config.MEAN
    #image /= config.STD
    image = np.transpose(image, (2, 0, 1))
    #image = np.expand_dims(image, 0)

    # return the preprocessed image
    return image
    

In [ ]:
# convert the labels to dict 
labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels.breed.value_counts()

In [ ]:
labels = dict(zip(labels.id, labels.breed))
#print(labels)

Convert the lables to numbers 

In [ ]:
label_names = list(set(labels.values()))
labels_num = my_dict = dict(zip(label_names, range(len(label_names))))
for key, value in labels.items():
    #print(labels[key])
    labels[key]= labels_num[value]


In [ ]:
#  display image 
img1 = "../input/dog-breed-identification/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg"
idx = labels[img1.split("/")[-1].split(".")[0]]
breed_label = list(labels_num.keys())[list(labels_num.values()).index(idx)]
plt_imshow(breed_label,img1)

In [ ]:
#next(iter(test_loader))

In [ ]:
class DogBreedsDataset(Dataset):
    """Dog Breeds dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_frame = pd.read_csv(csv_file)
        self.map = dict(zip(self.labels_frame['breed'].unique(),range(0,len(self.labels_frame['breed'].unique()))))
        self.labels_frame['breed'] = self.labels_frame['breed'].map(self.map)
        self.root_dir = root_dir
        self.transform = transform
        
    def getmap(self):
        return self.map
        
    def __getclasses__(self):
        return self.labels_frame['breed'].unique().tolist()

    def __len__(self):
        return len(self.labels_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.labels_frame.iloc[idx, 0])
        img_name = img_name + '.jpg'
        
        image = io.imread(img_name)
        PIL_image = Image.fromarray(image)
        label = self.labels_frame.iloc[idx, 1:]
        label = [int(label) for x in label]
        label = np.asarray(label)
        label = torch.from_numpy(label)
        
        if self.transform:
            image = self.transform(PIL_image)
        #sample = {'image': image, 'label': label}
        return image,label

In [ ]:
class DogBreedsTestDataset(Dataset):
    """Dog Breeds Test dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels_frame = pd.read_csv(csv_file)
        self.labels_frame = self.labels_frame[['id']]
        self.root_dir = root_dir
        self.transform = transform
   
    def __len__(self):
        return len(self.labels_frame)


    def __getitem__(self, idx):
        title = self.labels_frame.iloc[idx, 0]
        img_name = os.path.join(self.root_dir,
                                title)
        img_name = img_name + '.jpg'
        
        image = io.imread(img_name)
        PIL_image = Image.fromarray(image)
        # print(PIL_image)
        if self.transform:
            image = self.transform(PIL_image)
        sample = {'image': image, 'title': title}
        return image,title

In [ ]:
from sklearn.model_selection import train_test_split
data_dir = '../input/dog-breed-identification'
for_stratify = pd.read_csv("../input/dog-breed-identification/labels.csv")

# how many samples per batch to load
BATCH_SIZE = 20
# percentage of training set to use as validation
valid_size = 0.2

# TODO: Define transforms for the training data and testing data
transform = transforms.Compose([transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transforms = transforms.Compose([
                                      transforms.ToTensor()])


In [ ]:
train_data = DogBreedsDataset(csv_file='../input/dog-breed-identification/labels.csv',root_dir='../input/dog-breed-identification/train', transform=transform)
train_indices, val_indices = train_test_split(list(range(len(for_stratify.breed))), test_size=0.2, stratify=for_stratify.breed)

trainData = torch.utils.data.Subset(train_data, train_indices)
valData = torch.utils.data.Subset(train_data, val_indices)

trainDataLoader = DataLoader(trainData, shuffle=True,batch_size=BATCH_SIZE)
valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE)

In [ ]:
print(len(trainData))
print(len(valData))
print(len(train_data)==(len(trainData)+len(valData)), len(train_data))

In [ ]:
number_of_classes=len(np.unique(list(labels.values())))

In [ ]:
test_data = DogBreedsTestDataset(csv_file='../input/dog-breed-identification/sample_submission.csv',root_dir='../input/dog-breed-identification/test', transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20)

In [ ]:
from PIL import Image
from skimage import io, transform
#test_data[0]

In [ ]:
test_data[0]

In [ ]:
#pretraining 
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)

# print out the model structure
print(vgg16)

In [ ]:
"""#pretraining 
# Load the pretrained model from pytorch
vgg19 = models.vgg19(pretrained=True)

# print out the model structure
print(vgg19)"""

In [ ]:
# Freeze training for all "features" layers
for param in vgg16.features.parameters():
    param.requires_grad = False

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
n_inputs = vgg16.classifier[6].in_features

# add last linear layer (n_inputs -> 5 flower classes)
# new layers automatically have requires_grad = True
last_layer = nn.Linear(n_inputs, number_of_classes)

vgg16.classifier[6] = last_layer

# if GPU is available, move the model to GPU
if train_on_gpu:
    vgg16.cuda()

# check to see that your last layer produces the expected number of outputs
print(vgg16.classifier[6].out_features)
#print(vgg16)

In [ ]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.001
# optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001)

# ADAM optim
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.00001)

In [ ]:
feats_list = list(vgg16.features)

In [ ]:
new_feats_list = []
for feat in feats_list:
    new_feats_list.append(feat)
    if isinstance(feat, nn.Conv2d):
        new_feats_list.append(nn.Dropout(p=0.5, inplace=True))

# modify convolution layers
vgg16.features = nn.Sequential(*new_feats_list)

In [ ]:
vgg16

In [ ]:
from skimage import io, transform
from PIL import Image
# number of epochs to train the model
n_epochs = 5

loss_values = []
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    ###################
    # train the model #
    ###################
    # model by default is set to train
    
    for batch_i, (data, target) in enumerate(trainDataLoader):
        # move tensors to GPU if CUDA is available
       
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = vgg16(data)
        # calculate the batch loss
        loss = criterion(output, torch.max(target, 1)[1])
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss 
        train_loss += loss.item()
        running_loss += loss.item()
        
        if batch_i % 20 == 19:    # print training loss every specified number of mini-batches
            print('Epoch %d, Batch %d loss: %.16f' %
                  (epoch, batch_i + 1, train_loss / 20))
            
            train_loss = 0.0
            
    loss_values.append(running_loss / len(train_data))
    
plt.plot(loss_values)

In [ ]:
loss_values

In [ ]:
valid_loss = 0.0
vgg16.eval()
for batch_i, (data, target) in enumerate(valDataLoader):
        # move tensors to GPU if CUDA is available
       
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
            
        output = vgg16(data)
        # calculate the batch loss
        loss = criterion(output, torch.max(target, 1)[1])
        
        # update training loss 
        valid_loss += loss.item()
        
        if batch_i % 20 == 19:    # print validation loss every specified number of mini-batches
            print('Validation Loss Batch %d loss: %.16f' %
                  (batch_i + 1, valid_loss / 20))
            valid_loss = 0.0

In [ ]:
from skimage import io, transform
from PIL import Image
results = {}
vgg16.eval()
#test_data = DogBreedsTestset(csv_file='../input/dog-breed-identification/sample_submission.csv',root_dir='../input/dog-breed-identification/test', transform=transform)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=20)

for batch_i, (data, title) in enumerate(test_loader):
        # move tensors to GPU if CUDA is available
        images,titles = data, title
        #print(images.shape)
        #print(titles)
               
        if train_on_gpu:
            images = images.cuda()
        #print(title)
        logits = vgg16(images)
        output = torch.nn.functional.softmax(logits, dim=1)
        
        for k in range(len(titles)):
            name = titles[k]
            results[name] = output[k].cpu().tolist()

In [ ]:
output_df = pd.DataFrame(results).transpose()
inv_map = {v: k for k, v in train_data.getmap().items()}
output_df.rename(columns=inv_map,inplace=True)
output_df = output_df.reset_index()
output_df.rename(columns={'index':'id'},inplace=True)
output_df.to_csv('output2.csv',index=False)

https://www.kaggle.com/carloalbertobarbano/vgg16-transfer-learning-pytorch